In [1]:
pip install awscli boto3 s3 s3fs setfit

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import pandas as pd
import s3fs
import boto3

aws_access_key_id = getpass.getpass("Enter your AWS Access Key ID: ")
aws_secret_access_key = getpass.getpass("Enter your AWS Secret Access Key: ")

s3r = boto3.resource(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

In [3]:
fs = s3fs.S3FileSystem()
bucket_name = "vgsa"

file_path = "s3://vgsa/silver/silver_tables_dbt_result/games_sentiment_analysis_silver/top_selling_games_reviews/4488e75a-9522-4ad0-9dee-1e3d32d603e0/20251009_143214_00071_275s7_7642792d-3177-448a-b15b-d53156c8d51e"
df = pd.read_parquet(file_path,filesystem=fs)

In [4]:
import os
os.environ['HF_TOKEN'] = getpass.getpass("Enter your HF Token ID: ")

In [ ]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="auto",
    api_key=os.environ["HF_TOKEN"])

/Users/ameedahmed/Desktop/my_lambda_docker/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
unique_games = df["name"].unique()
empty_df = pd.DataFrame()
for game in unique_games:
  game_df = df[df["name"]==game]
  game_df = game_df.reset_index(drop=True)
  #for each game cycle through the reviews row by row
  for i in range(len(game_df)):
    try:
      if len(game_df["review"].iloc[i])>512:
        result = client.text_classification(game_df["review"].iloc[i][:512],model="tabularisai/multilingual-sentiment-analysis")
        print(result)
      else:
        result = client.text_classification((game_df["review"].iloc[i]),model="tabularisai/multilingual-sentiment-analysis")
        print(result)
      max_result = max(result,key=lambda x: x["score"])
      game_df.loc[i,"sentiment"] = max_result["label"]
    except Exception as e:
        print(game_df.iloc[i])
        print("Error",e)
  ##Once the rows have been populated, append this dataframe to the empty dataframe
  empty_df = pd.concat([empty_df,game_df],ignore_index=True)

[TextClassificationOutputElement(label='Very Positive', score=0.47661980986595154), TextClassificationOutputElement(label='Positive', score=0.36738821864128113), TextClassificationOutputElement(label='Neutral', score=0.1032700464129448), TextClassificationOutputElement(label='Negative', score=0.028763512149453163), TextClassificationOutputElement(label='Very Negative', score=0.023958398029208183)]
[TextClassificationOutputElement(label='Positive', score=0.5687246322631836), TextClassificationOutputElement(label='Neutral', score=0.2748916447162628), TextClassificationOutputElement(label='Very Positive', score=0.10861106961965561), TextClassificationOutputElement(label='Negative', score=0.02835247293114662), TextClassificationOutputElement(label='Very Negative', score=0.019420096650719643)]
[TextClassificationOutputElement(label='Very Positive', score=0.6060214042663574), TextClassificationOutputElement(label='Positive', score=0.2865768373012543), TextClassificationOutputElement(label='N

KeyboardInterrupt: 

In [7]:
empty_df

""


In [ ]:
path = "s3://vgsa/gold/sentiment_analysis_result/sentiment.parquet"
fs = s3fs.S3FileSystem(
    key=aws_access_key_id,
    secret=aws_secret_access_key)
empty_df.to_parquet(path,filesystem=fs,index=False)

In [5]:
import boto3
import pandas as pd
import s3fs
import os
from huggingface_hub import InferenceClient

s3 = boto3.client("s3")
client = boto3.client("secretsmanager")
fs = s3fs.S3FileSystem()

bucket_name = "vgsa"
file_path = "s3://vgsa/silver/silver_tables_dbt_result/games_sentiment_analysis_silver/top_selling_games_reviews/4488e75a-9522-4ad0-9dee-1e3d32d603e0/20251009_143214_00071_275s7_7642792d-3177-448a-b15b-d53156c8d51e"
df = pd.read_parquet(file_path,filesystem=fs)

#Read the hugginface token
secret_name = "hf_token"
response = client.get_secret_value(SecretId=secret_name)
hf_token = response["SecretString"]
os.environ["HF_TOKEN"] = hf_token
client = InferenceClient(provider="auto",api_key=os.environ["HF_TOKEN"])
# Load model
batch_size=20
df["review_new"] = df["review"].apply(lambda x: x[:512] if len(x)>512 else x)

In [8]:
##Now perform classification in batches
for a in range(0, len(df), batch_size):
    batch = df["review_new"].iloc[a:a+batch_size].tolist()
    results = client.text_classification(batch, model="tabularisai/multilingual-sentiment-analysis")
    results_label = [i["label"] for i in results]
    df.loc[a:a+batch_size-1,"sentiment"] = results_label